# Building Your Own Mini Agentic Framework

This notebook breaks down how to create a lightweight agentic framework from scratch. We'll build each component step-by-step with clear explanations.

## What is an Agentic Framework?

An agentic framework provides:
1. **Message handling** - Structured communication between user, agent, and tools
2. **Tool system** - Functions the agent can call to interact with the world
3. **Memory management** - Conversation history and context retention
4. **Agent execution loop** - The core decision-making and execution cycle
5. **Model abstraction** - Interface to different LLM providers

Let's build each piece!


In [ ]:
# Initial setup - run this first!
# All necessary imports for the framework

from dataclasses import dataclass
from enum import Enum
from typing import Optional, Dict, Any, List, Callable
from datetime import datetime
from abc import ABC, abstractmethod
import inspect
import json

print("✅ All imports loaded successfully!")


## Part 1: Message Types

The foundation of any agent framework is message passing. Messages represent the conversation between user, agent, and tools.


In [ ]:
class MessageRole(Enum):
    """Defines who sent the message in the conversation"""
    USER = "user"
    ASSISTANT = "assistant"
    SYSTEM = "system"
    TOOL = "tool"

@dataclass
class Message:
    """
    Represents a single message in the conversation.
    
    Attributes:
        role: Who sent the message (user, assistant, system, or tool)
        content: The actual message text
        name: Optional name for tool calls/results
        tool_call_id: Links tool results back to tool calls
        timestamp: When the message was created
    """
    role: MessageRole
    content: str
    name: Optional[str] = None
    tool_call_id: Optional[str] = None
    timestamp: datetime = None
    
    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = datetime.now()
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert message to dictionary format for LLM API calls"""
        msg = {"role": self.role.value, "content": self.content}
        if self.name:
            msg["name"] = self.name
        if self.tool_call_id:
            msg["tool_call_id"] = self.tool_call_id
        return msg

# Example usage
user_msg = Message(MessageRole.USER, "What's the weather in New York?")
print(f"User message: {user_msg.content}")
print(f"Formatted: {user_msg.to_dict()}")


## Part 2: Tool System

Tools are functions that agents can call to interact with external systems. Each tool needs:
- A name and description (for the LLM to understand when to use it)
- Parameters (what inputs it needs)
- The actual function to execute


In [ ]:
@dataclass
class Tool:
    """
    Wraps a Python function as a tool the agent can use.
    
    The tool description and parameter info help the LLM decide:
    1. When to call this tool
    2. What arguments to pass
    
    Attributes:
        name: Unique identifier for the tool
        description: Human-readable explanation of what the tool does
        func: The actual Python function to execute
        parameters: JSON schema describing the function parameters
    """
    name: str
    description: str
    func: Callable
    parameters: Dict[str, Any]
    
    @classmethod
    def from_function(cls, func: Callable, name: str = None, description: str = None):
        """
        Automatically create a Tool from a Python function.
        Uses function signature and docstring to infer parameters.
        """
        sig = inspect.signature(func)
        name = name or func.__name__
        description = description or func.__doc__ or ""
        
        # Extract parameter schema from function signature
        properties = {}
        required = []
        
        for param_name, param in sig.parameters.items():
            if param_name == "self":
                continue
                
            param_type = "string"  # default
            if param.annotation != inspect.Parameter.empty:
                if param.annotation == int:
                    param_type = "integer"
                elif param.annotation == float:
                    param_type = "number"
                elif param.annotation == bool:
                    param_type = "boolean"
                elif param.annotation == list:
                    param_type = "array"
            
            properties[param_name] = {"type": param_type}
            if param.default == inspect.Parameter.empty:
                required.append(param_name)
        
        parameters = {
            "type": "object",
            "properties": properties,
            "required": required
        }
        
        return cls(name=name, description=description, func=func, parameters=parameters)
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert tool to OpenAI function calling format"""
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": self.parameters
            }
        }
    
    async def call(self, **kwargs) -> Any:
        """Execute the tool with given arguments"""
        if inspect.iscoroutinefunction(self.func):
            return await self.func(**kwargs)
        else:
            return self.func(**kwargs)

# Example: Create a simple weather tool
def get_temperature(city: str) -> str:
    """Get the current temperature for a city
    
    Args:
        city: The name of the city (e.g., "New York", "London")
    
    Returns:
        Temperature string like "22°C"
    """
    temperatures = {
        "New York": "22°C",
        "London": "15°C",
        "Tokyo": "18°C",
        "Paris": "20°C"
    }
    return temperatures.get(city, "Unknown")

# Create tool from function
weather_tool = Tool.from_function(get_temperature)
print(f"Tool name: {weather_tool.name}")
print(f"Tool description: {weather_tool.description}")
print(f"Tool schema:\n{weather_tool.to_dict()}")


## Part 3: Memory Management

Agents need to remember past conversations. We'll implement a simple memory system that:
- Stores message history
- Provides context for the LLM
- Allows retrieval and filtering


In [ ]:
class Memory:
    """
    Manages conversation history and context for the agent.
    
    This is a simple in-memory storage. In production, you might want:
    - Persistent storage (database)
    - Vector embeddings for semantic search
    - Token limits and summarization
    - Session management
    
    Attributes:
        messages: List of all messages in the conversation
        max_messages: Maximum number of messages to keep (None = unlimited)
    """
    
    def __init__(self, max_messages: Optional[int] = None):
        self.messages: List[Message] = []
        self.max_messages = max_messages
    
    def add_message(self, message: Message):
        """Add a message to memory"""
        self.messages.append(message)
        
        # Trim old messages if we exceed max
        if self.max_messages and len(self.messages) > self.max_messages:
            # Keep system messages and recent messages
            system_msgs = [m for m in self.messages if m.role == MessageRole.SYSTEM]
            recent_msgs = self.messages[-self.max_messages:]
            self.messages = system_msgs + recent_msgs[len(system_msgs):]
    
    def get_messages(self, include_system: bool = True) -> List[Message]:
        """
        Get all messages, optionally filtering out system messages.
        
        System messages are usually kept in the conversation but
        you might want to exclude them for some operations.
        """
        if include_system:
            return self.messages.copy()
        return [m for m in self.messages if m.role != MessageRole.SYSTEM]
    
    def get_context_messages(self) -> List[Dict[str, Any]]:
        """Get messages formatted for LLM API calls"""
        return [msg.to_dict() for msg in self.messages]
    
    def clear(self):
        """Clear all messages except system messages"""
        system_msgs = [m for m in self.messages if m.role == MessageRole.SYSTEM]
        self.messages = system_msgs
    
    def get_recent(self, n: int) -> List[Message]:
        """Get the last N messages"""
        return self.messages[-n:] if n > 0 else []

# Example usage
memory = Memory(max_messages=10)
memory.add_message(Message(MessageRole.SYSTEM, "You are a helpful assistant."))
memory.add_message(Message(MessageRole.USER, "Hello!"))
memory.add_message(Message(MessageRole.ASSISTANT, "Hi there! How can I help you?"))

print(f"Total messages: {len(memory.get_messages())}")
print(f"Context for LLM: {len(memory.get_context_messages())} messages")


## Part 4: Model Abstraction

We need a way to interact with different LLM providers (OpenAI, Ollama, etc.).
This abstraction layer lets us swap models without changing the agent code.


In [ ]:
class LLMProvider(ABC):
    """
    Abstract base class for LLM providers.
    
    This defines the interface that all model providers must implement.
    By abstracting this, we can support multiple providers:
    - OpenAI
    - Ollama
    - Anthropic Claude
    - Local models
    - etc.
    """
    
    @abstractmethod
    async def chat_completion(
        self,
        messages: List[Dict[str, Any]],
        tools: Optional[List[Tool]] = None,
        temperature: float = 0.7,
        max_tokens: Optional[int] = None
    ) -> Dict[str, Any]:
        """
        Generate a chat completion from the LLM.
        
        Args:
            messages: List of message dicts in OpenAI format
            tools: Optional list of tools the model can use
            temperature: Sampling temperature (0-2)
            max_tokens: Maximum tokens to generate
            
        Returns:
            Dict with 'content' and optional 'tool_calls'
        """
        pass


class OpenAIProvider(LLMProvider):
    """
    OpenAI API provider implementation.
    
    Uses the OpenAI client to make chat completion requests.
    Handles tool calling if tools are provided.
    """
    
    def __init__(self, model: str = "gpt-4o-mini", api_key: Optional[str] = None, base_url: Optional[str] = None):
        from openai import OpenAI
        
        self.model = model
        self.client = OpenAI(api_key=api_key, base_url=base_url)
    
    async def chat_completion(
        self,
        messages: List[Dict[str, Any]],
        tools: Optional[List[Tool]] = None,
        temperature: float = 0.7,
        max_tokens: Optional[int] = None
    ) -> Dict[str, Any]:
        """Call OpenAI API"""
        kwargs = {
            "model": self.model,
            "messages": messages,
            "temperature": temperature,
        }
        
        if max_tokens:
            kwargs["max_tokens"] = max_tokens
        
        if tools:
            kwargs["tools"] = [tool.to_dict() for tool in tools]
            kwargs["tool_choice"] = "auto"
        
        response = self.client.chat.completions.create(**kwargs)
        
        message = response.choices[0].message
        result = {"content": message.content}
        
        # Extract tool calls if present
        if message.tool_calls:
            result["tool_calls"] = []
            for tool_call in message.tool_calls:
                result["tool_calls"].append({
                    "id": tool_call.id,
                    "name": tool_call.function.name,
                    "arguments": json.loads(tool_call.function.arguments)
                })
        
        return result


class OllamaProvider(LLMProvider):
    """
    Ollama provider implementation.
    
    Works with local Ollama models via OpenAI-compatible API.
    """
    
    def __init__(self, model: str = "llama3", base_url: str = "http://localhost:11434/v1", api_key: str = "ollama"):
        from openai import OpenAI
        
        self.model = model
        self.client = OpenAI(base_url=base_url, api_key=api_key)
    
    async def chat_completion(
        self,
        messages: List[Dict[str, Any]],
        tools: Optional[List[Tool]] = None,
        temperature: float = 0.7,
        max_tokens: Optional[int] = None
    ) -> Dict[str, Any]:
        """Call Ollama via OpenAI-compatible API"""
        # Note: Tool calling support may vary by Ollama model
        kwargs = {
            "model": self.model,
            "messages": messages,
            "temperature": temperature,
        }
        
        if max_tokens:
            kwargs["max_tokens"] = max_tokens
        
        if tools:
            kwargs["tools"] = [tool.to_dict() for tool in tools]
        
        response = self.client.chat.completions.create(**kwargs)
        
        message = response.choices[0].message
        result = {"content": message.content}
        
        # Extract tool calls if present
        if hasattr(message, 'tool_calls') and message.tool_calls:
            result["tool_calls"] = []
            for tool_call in message.tool_calls:
                result["tool_calls"].append({
                    "id": tool_call.id,
                    "name": tool_call.function.name,
                    "arguments": json.loads(tool_call.function.arguments)
                })
        
        return result

print("Model providers defined!")
print("- OpenAIProvider: For OpenAI API")
print("- OllamaProvider: For local Ollama models")


## Part 5: The Agent - Core Execution Loop

This is the heart of the framework. The agent:
1. Receives user input
2. Sends messages + tools to LLM
3. If LLM wants to call a tool, execute it
4. Send tool result back to LLM
5. Return final response

This is the "ReAct" pattern: Reasoning + Acting


In [ ]:
class Agent:
    """
    The main agent class that orchestrates the execution loop.
    
    This implements a simple ReAct (Reasoning + Acting) pattern:
    1. User sends message
    2. Agent thinks (LLM generates response)
    3. If tool call needed, agent acts (executes tool)
    4. Agent observes (gets tool result)
    5. Agent thinks again with new information
    6. Returns final response
    
    Attributes:
        provider: The LLM provider to use
        tools: List of available tools
        memory: Conversation memory/history
        system_prompt: Instructions for the agent
        max_iterations: Max tool call iterations (prevents infinite loops)
    """
    
    def __init__(
        self,
        provider: LLMProvider,
        tools: Optional[List[Tool]] = None,
        memory: Optional[Memory] = None,
        system_prompt: Optional[str] = None,
        max_iterations: int = 10
    ):
        self.provider = provider
        self.tools = {tool.name: tool for tool in (tools or [])}
        self.memory = memory or Memory()
        self.max_iterations = max_iterations
        
        # Add system prompt if provided
        if system_prompt:
            self.memory.add_message(Message(MessageRole.SYSTEM, system_prompt))
    
    async def run(self, user_input: str, reset_context: bool = False) -> str:
        """
        Main execution method - processes user input and returns response.
        
        This implements the agent loop:
        1. Add user message to memory
        2. Get context messages
        3. Call LLM with messages + tools
        4. If tool calls, execute them and loop back
        5. Return final response
        
        Args:
            user_input: The user's message
            reset_context: If True, clear conversation history first
            
        Returns:
            Final response string from the agent
        """
        if reset_context:
            self.memory.clear()
        
        # Add user message
        user_msg = Message(MessageRole.USER, user_input)
        self.memory.add_message(user_msg)
        
        iteration = 0
        while iteration < self.max_iterations:
            iteration += 1
            
            # Get current context
            messages = self.memory.get_context_messages()
            
            # Call LLM
            response = await self.provider.chat_completion(
                messages=messages,
                tools=list(self.tools.values()) if self.tools else None
            )
            
            # Handle tool calls
            if response.get("tool_calls"):
                # Add assistant message with tool calls to memory
                assistant_msg = Message(MessageRole.ASSISTANT, response.get("content", ""))
                self.memory.add_message(assistant_msg)
                
                # Execute each tool call
                for tool_call in response["tool_calls"]:
                    tool_id = tool_call["id"]
                    tool_name = tool_call["name"]
                    tool_args = tool_call["arguments"]
                    
                    # Find and execute tool
                    if tool_name in self.tools:
                        tool = self.tools[tool_name]
                        try:
                            tool_result = await tool.call(**tool_args)
                            result_str = str(tool_result) if not isinstance(tool_result, str) else tool_result
                        except Exception as e:
                            result_str = f"Error: {str(e)}"
                    else:
                        result_str = f"Tool '{tool_name}' not found"
                    
                    # Add tool result to memory
                    tool_msg = Message(
                        MessageRole.TOOL,
                        result_str,
                        name=tool_name,
                        tool_call_id=tool_id
                    )
                    self.memory.add_message(tool_msg)
                
                # Continue loop to let LLM process tool results
                continue
            
            # No tool calls - we have the final answer
            content = response.get("content", "")
            if content:
                assistant_msg = Message(MessageRole.ASSISTANT, content)
                self.memory.add_message(assistant_msg)
                return content
        
        # Max iterations reached
        return "Error: Maximum iterations reached. The agent may be stuck in a loop."

print("Agent class defined!")
print("The agent handles the complete execution loop automatically.")


## Part 6: Putting It All Together - Example Usage

Let's create a working example that demonstrates the full framework!


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Example 1: Create tools
def get_temperature(city: str) -> str:
    """Get the current temperature for a city"""
    temperatures = {
        "New York": "22°C",
        "London": "15°C",
        "Tokyo": "18°C",
        "Paris": "20°C"
    }
    return temperatures.get(city, "Unknown")

def calculate(expression: str) -> str:
    """Evaluate a mathematical expression
    
    Args:
        expression: A math expression like "2 + 2" or "10 * 5"
    
    Returns:
        The result as a string
    """
    try:
        # Safety: Only allow basic math operations
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

# Create tools
tools = [
    Tool.from_function(get_temperature),
    Tool.from_function(calculate, description="Performs basic math calculations")
]

print("Tools created:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")


In [ ]:
# Example 2: Create agent with OpenAI
# Uncomment to use OpenAI (requires API key in .env)

# provider = OpenAIProvider(
#     model="gpt-4o-mini",
#     api_key=os.getenv("OPENAI_API_KEY")
# )

# agent = Agent(
#     provider=provider,
#     tools=tools,
#     system_prompt="You are a helpful assistant. Use tools when appropriate.",
#     max_iterations=5
# )

# # Test the agent
# result = await agent.run("What's the temperature in New York and what's 15 * 8?")
# print(f"Agent response: {result}")


In [ ]:
# Example 3: Create agent with Ollama (if you have it running locally)
# Adjust the base_url to match your Ollama instance

# provider = OllamaProvider(
#     model="llama3",
#     base_url="http://localhost:11434/v1"
# )

# agent = Agent(
#     provider=provider,
#     tools=tools,
#     system_prompt="You are a helpful assistant. Use tools when appropriate.",
#     max_iterations=5
# )

# # Test the agent
# result = await agent.run("What's the temperature in Tokyo?")
# print(f"Agent response: {result}")


## Part 7: Advanced Features - What to Add Next

Here are some enhancements you might want to add:


In [ ]:
# 1. Streaming Responses
# Instead of waiting for complete response, stream tokens as they're generated

class StreamingAgent(Agent):
    """Agent that supports streaming responses"""
    
    async def run_stream(self, user_input: str):
        """Generator that yields tokens as they're generated"""
        user_msg = Message(MessageRole.USER, user_input)
        self.memory.add_message(user_msg)
        
        # Implementation would use provider's streaming API
        # This is a placeholder showing the pattern
        messages = self.memory.get_context_messages()
        
        # In real implementation, you'd stream from provider:
        # async for chunk in provider.chat_completion_stream(...):
        #     yield chunk
        pass

# 2. Function Calling with Pydantic Models
from pydantic import BaseModel

class WeatherRequest(BaseModel):
    city: str
    unit: str = "celsius"

# Tools can use Pydantic for validation:
# tool = Tool.from_pydantic(WeatherRequest, func=get_weather)

# 3. Memory with Vector Search
# Store messages with embeddings, search by semantic similarity
class VectorMemory(Memory):
    """Memory with vector search capabilities"""
    
    def __init__(self):
        super().__init__()
        self.embeddings = []  # Would store vector embeddings
    
    def search_similar(self, query: str, limit: int = 5):
        """Find similar past conversations"""
        # Implementation would:
        # 1. Embed the query
        # 2. Find similar message embeddings
        # 3. Return relevant context
        pass

# 4. Multi-Agent Systems
class MultiAgentSystem:
    """Orchestrates multiple agents working together"""
    
    def __init__(self, agents: Dict[str, Agent]):
        self.agents = agents
    
    async def route(self, user_input: str) -> str:
        """Route user input to appropriate agent"""
        # Could use a router agent to decide which specialist to use
        # Or implement rule-based routing
        pass

# 5. Observability & Logging
class ObservableAgent(Agent):
    """Agent with built-in logging and metrics"""
    
    async def run(self, user_input: str, **kwargs):
        # Log input
        print(f"[INPUT] {user_input}")
        
        # Track metrics
        start_time = datetime.now()
        result = await super().run(user_input, **kwargs)
        duration = (datetime.now() - start_time).total_seconds()
        
        # Log output
        print(f"[OUTPUT] {result}")
        print(f"[METRICS] Duration: {duration}s, Messages: {len(self.memory.messages)}")
        
        return result

print("Advanced patterns shown above:")
print("1. Streaming responses")
print("2. Pydantic model validation")
print("3. Vector memory search")
print("4. Multi-agent orchestration")
print("5. Observability & logging")


## Part 8: Framework Architecture Summary

Here's how all the pieces fit together:

```
┌─────────────────────────────────────────────────────────┐
│                     User Input                          │
└───────────────────┬─────────────────────────────────────┘
                    │
                    ▼
        ┌───────────────────────┐
        │      Agent.run()      │
        └───────────┬───────────┘
                    │
        ┌───────────▼───────────┐
        │   Memory.add_message  │  ← Stores conversation
        └───────────┬───────────┘
                    │
        ┌───────────▼───────────┐
        │  Get context messages │  ← Retrieves history
        └───────────┬───────────┘
                    │
        ┌───────────▼───────────┐
        │  LLMProvider.chat()   │  ← Calls LLM API
        │  + Tools schema        │
        └───────────┬───────────┘
                    │
        ┌───────────▼───────────┐
        │   Tool calls?         │
        └───┬───────────────┬───┘
            │ Yes           │ No
            ▼               ▼
    ┌───────────────┐   ┌──────────────┐
    │ Execute Tool  │   │ Return Final │
    │ Store Result  │   │   Response   │
    └───────┬───────┘   └──────────────┘
            │
            └───► Loop back to LLM
```

### Key Components:

1. **Message**: Data structure for conversation
2. **Tool**: Wrapper for functions agent can call
3. **Memory**: Manages conversation history
4. **LLMProvider**: Abstracts model API calls
5. **Agent**: Orchestrates the execution loop

### The Loop:

1. User sends message → stored in memory
2. Agent gets context → sends to LLM
3. LLM responds → might call tools
4. Tools execute → results stored
5. Loop back to step 2 with tool results
6. LLM gives final answer → return to user


## Part 9: Comparison with Existing Frameworks

### How this compares to:

**Pydantic AI** (which you've been using):
- ✅ Similar concepts (Agent, Tools, Messages)
- ✅ This is more minimal and educational
- ✅ Pydantic AI has more features (validation, streaming, etc.)

**LangChain**:
- ✅ LangChain is much larger with many integrations
- ✅ This framework is ~200 lines vs LangChain's thousands
- ✅ Good for learning, LangChain for production

**AutoGen/CrewAI**:
- ✅ Those focus on multi-agent coordination
- ✅ This is a single-agent framework
- ✅ Can extend this to multi-agent later

### When to use this framework:
- 🎓 Learning how agents work internally
- 🚀 Building custom agentic workflows
- 🛠️ Simple use cases where full frameworks are overkill
- 🧪 Prototyping before committing to a framework

### When to use existing frameworks:
- 📦 Production applications (use Pydantic AI, LangChain)
- 🌐 Need many integrations (use LangChain)
- 👥 Multi-agent systems (use AutoGen, CrewAI)
- 🔧 Complex tool orchestration (use specialized frameworks)


## Part 10: Next Steps & Extensions

### Ideas for extending this framework:

1. **Persistent Memory**: Save conversations to a database
2. **Tool Caching**: Cache tool results to avoid redundant calls
3. **Error Handling**: Better error recovery and retry logic
4. **Cost Tracking**: Monitor token usage and costs
5. **Rate Limiting**: Prevent too many API calls
6. **Structured Output**: Use Pydantic models for agent responses
7. **Planner Agents**: Add planning/reflection capabilities
8. **Workflow Orchestration**: Chain multiple agents together
9. **Observability**: Add tracing, logging, metrics
10. **Configuration**: YAML/JSON config files for agents

### Try building:
- A research agent that searches the web
- A coding assistant that can read/write files
- A data analysis agent that queries databases
- A customer service agent with knowledge base
